In [3]:
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


file_path = "Processed_Data/Mapped_Dataset.csv"

def data_generator(file_path, batch_size=512, num_classes=10):
    chunks = pd.read_csv(file_path, chunksize=batch_size)
    
    for chunk in chunks:
        X = chunk.iloc[:, :-1].values.reshape(-1, 46, 1)  # Định dạng input
        y = chunk.iloc[:, -1].values
        y_onehot = to_categorical(y, num_classes=num_classes)
        yield X, y_onehot

# Tạo callback -> kết thúc epoch thì regenerate train_gen        
from tensorflow.keras.callbacks import Callback

class ResetGeneratorCallback(Callback):
    def __init__(self, file_path, batch_size=512, num_classes=10):
        self.file_path = file_path
        self.batch_size = batch_size
        self.num_classes = num_classes

    def on_epoch_end(self, epoch, logs=None):
        global train_gen  # Cập nhật biến toàn cục
        print(f"\nEpoch {epoch+1} kết thúc, tạo generator mới...\n")
        train_gen = data_generator(self.file_path, self.batch_size, self.num_classes)  # Reset generator


In [ ]:
train_gen = data_generator(file_path, batch_size=512, num_classes=10)

from tensorflow import keras
# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường

model = keras.Sequential([
    layers.Input(shape=(46,1)),
    layers.Conv1D(filters=32, kernel_size=3, padding="same",activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0,5),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Reset gen
reset_callback = ResetGeneratorCallback(file_path, batch_size=512, num_classes=10)
model.fit(train_gen, epochs=10, callbacks = [reset_callback])

# Lưu mô hình
model.save("cnn_model.h5")

Epoch 1/10
91084/91084 [==============================] - 1591s 17ms/step - loss: 0.5323 - accuracy: 0.8016
Epoch 2/10
91084/91084 [==============================] - 0s 0us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
